# PREPARE THE DATA 

In [2]:
#1 Importing needed librairies 

import pandas as pd
import os
import numpy as np

In [3]:
#2 Read the csv file
df = pd.read_csv('/Users/mac/Downloads/shark.csv', sep=',', engine='python')

# ELEMENTARY CLEANING 

In [4]:
#3 Create a copy of the original dataframe to modify it safely
# Drop the columns [ 'pdf' 'href' 'formula' 'href' 'Case Number.1' 'Case Number.2' 'original order' 'Unnamed: 22' 'Unnamed: 23'] because they don't give any relevant information

table_shark = df.copy()
table_shark = table_shark.drop(table_shark.columns[16:], axis=1) 
#table_shark

In [5]:
#4 Clean the names of the remaining columns, drop all the spaces and return lowercase strings

table_shark.columns=[col.replace(' ', '').lower()  for col in table_shark.columns]
# Rename the columns:'casenumber' as 'id' ,'investigatororsource' as 'source' and 'fatal(y/n)' as 'fatalyn', to make the manipulation of the dataframe easier
table_shark = table_shark.rename(columns={'casenumber': 'id', 'investigatororsource': 'source', 'fatal(y/n)': 'fatalyn'})
#table_shark



In [6]:
#5 Dropping columns with high pencentage of nan values

null_cols = table_shark.isnull().sum()
null_cols
drop_percentage = round(null_cols[null_cols>0]/table_shark.shape[0]*100,2)
#drop_percentage

# No percentage higher than 75%, didn't drop any accroding to this criteria
#drop_cols=drop_percentage[drop_percentage>75].index
#drop_cols

In [7]:
#6 Drop the columns 'name', 'sex' and 'age' because don't give any relevant information
table_shark= table_shark.drop(['name', 'sex', 'age'], axis=1)

In [8]:
#7 Drop the rows containing NAN values
table_shark = table_shark.dropna()
table_shark= table_shark.reset_index(drop=True)

# CLEANING COLUMN BY COLUMN 

# Attribute "fatalyn"

In [9]:
#8 Cleaning values of "fatalyn" (drop the spaces from strings)
table_shark['fatalyn'] = table_shark['fatalyn'].str.replace(" ","")
table_shark.fatalyn.value_counts()

N    1433
Y     263
Name: fatalyn, dtype: int64

# Attribute "time"

In [10]:
#9 Cleaning the column corresponding to the attribute "time", the aim is to keep the hour when the incident happens 
import re
liste=[]
timelist=[]

for t in table_shark.time:
    a = re.findall(r'\d+', t)
    if a!=[]:
        liste.append(a)
    else:
        liste.append(t)

In [11]:
#10 split the day into several parts and make them corresponding with the hours
#Make all the values lowercase so as to standardize the processes

for i in range(len(liste)):
        
    if isinstance(liste[i], str)==True:
        
        if liste[i] in ['early morning', 'morning', 'late morning', 'early afternoon', 'afternoon',
                        'late afternoon','early evening', 'evening', 'night']:
            timelist.append(liste[i])
        else:
            timelist.append('NAN')
    else: 
        
        if 5<=int(liste[i][0])<=8 :
            timelist.append('early morning')
            
        elif 8<int(liste[i][0])<=11 :
            timelist.append('morning')
            
        elif 11<int(liste[i][0])<=12 :
            timelist.append('late morning')
                            
        elif 12<int(liste[i][0])<=15 :
            timelist.append('early afternoon')
                            
        elif 15<int(liste[i][0])<=16 :
            timelist.append('afternoon')
                            
        elif 15<int(liste[i][0])<=17 :
            timelist.append('late afternoon')
                            
        elif 17<int(liste[i][0])<=19 :
            timelist.append('early evening')
                            
        elif 19<int(liste[i][0])<=21 :
            timelist.append('evening')  
                            
        elif 21<int(liste[i][0])<=23 or 0<int(liste[i][0])<=4 :
            timelist.append('night') 
        else:
            timelist.append('NAN')
        
timelist = [element.replace(' ', '').lower()  for element in timelist]     

In [12]:
#11 Update the "time" columns by dropping the column and creating a new one with our new values 
n = table_shark.columns[10]
table_shark.drop(n, axis = 1, inplace = True) #(Drop that column)

In [13]:
table_shark[n] = timelist
table_shark = table_shark.replace('nan',np.NaN)  #Put the new serie in the column 

In [14]:
#12 Drop the NAN rows 
table_shark = table_shark.dropna()
table_shark= table_shark.reset_index(drop=True)

In [15]:
#Verify whether the cleaning succeded 
#table_shark.time.value_counts()

# Attribute "injury"

In [16]:
#13 Transforme "injury" values to lowercase
table_shark.injury = [element.lower()  for element in table_shark.injury] 

In [17]:
#14 Clusterize the column "injury" values into 5 cases: "death", "no injury", "provoked", "injury", "amputation"
# Make a list of the words that apprear in the column "injury" values the make them correspond with the 5 cases..
a= ['death', 'dead', 'fatal']
b= ['no', 'injury']
c= ['provoked']
d= ['laceration', 'lacerations','ambrasion','gash','bite','bruise', 'cut','bruised','abrasions','punctures', 'lacerated', 'injury','punctured', 'injured', 'injuries', 'bitten', 'wound', 'wounds', 'bumped' ]
e= ['severed', 'amputated']
liste_injury=[]
liste=[]
for t in table_shark.injury: 
    a = re.findall(r'\w+',t) # Split all the words 
    liste_injury.append(a)
# If any of the keys words (a,b,c,d,e) are in the list (liste_injury) of the words contained in the column "injury" values 
#then we append the case to the new list that we will put in the new column cleaned
for i in range(len(liste_injury)):
    result1 =  any(elem in a  for elem in liste_injury[i])
    result2 =  all(elem in b  for elem in liste_injury[i])
    result3 =  any(elem in c  for elem in liste_injury[i])
    result4 =  any(elem in d  for elem in liste_injury[i])
    result5 =  any(elem in e  for elem in liste_injury[i])
    if result1:
        liste.append('death') 
    elif result2:
        liste.append('no injury') 
    elif result3:
        liste.append('provoked') 
    elif result4:
        liste.append('injury') 
    elif result5:
        liste.append('amputation') 
    else: 
        liste.append(np.NaN)   

In [18]:
#15 Drop the column "injury" and create the cleaned one from our list "liste"
n = table_shark.columns[8]
table_shark.drop(n, axis = 1, inplace = True)

In [19]:
#16 Update our dataframe with the new column
table_shark[n] = liste

In [20]:
#17 Drop the NaN rows 
table_shark = table_shark.dropna()
table_shark= table_shark.reset_index(drop=True)
#table_shark

In [21]:
table_shark.injury.value_counts()

injury        818
death         519
provoked       34
amputation     18
no injury      14
Name: injury, dtype: int64

# Attribute "country"

In [22]:
#18 We create a list of all the countries and territories contained in each ocean worldwild
# and put every country written in the "country" column into the right ocean
Pacific_ocean = ['Australia', 'Brunei', 'Cambodia','Canada','Chile','China','Columbia','Costa Rica','Ecuador','El Salvador','Federated', 'Micronesia', 'Fiji','Guatemala',
'Honduras','Indonesia','Japan','Kiribati', 'North Korea','South Korea', 'Malaysia',  'Marshall islands', 'Islands,Mexico' ,'Nauru', 'New Zealand', 'Nicaragua','Palau', 'Panama', 'Papua', 
'papua New Guinea', 'Peru', 'Philippines', 'Russia', 'Samoa', 'Singapore', 'Solomon Islands', 'Taiwan', 'Thailand', 'Timor-Leste', 'Tonga',
'Tuvalu', 'United States', 'Vanuatu', 'Vietnam','American Samoa', 'Baker Island' ,'Cook Islands' ,'Coral Sea Islands' ,'Easter Island' ,'French Polynesia',
'Guam' ,'Hong Kong','Howland Island' ,'Jarvis Island' ,'Johnston Island' ,'Kingman Reef' ,'Macau' ,'Midway Atoll','New Caledonia' ,'Niue' ,
'Norfolk Island' ,'Northern Mariana Islands' ,'Palmyra Atoll','Pitcairn Islands' ,'Tokelau' ,'Wallis and Futuna' ,'Wake Island' ]
Pacific_ocean = [element.lower()  for element in Pacific_ocean] 
Atlantic_ocean = ['Bulgaria','Croatia','Cyprus','Denmark','Estonia','Finland','France','Georgia','Germany','Greece','Iceland','Ireland','Italy','Latvia','Lithuania','Malta',
'Monaco','Montenegro','Netherlands','Northern Cyprus','Norway','Poland','Portugal','Romania','Russia','Slovenia','Spain','Sweden','Turkey',
'Ukraine','United Kingdom','Faroe Islands','Gibraltar','Guernsey','Isle of Man','Jersey','Algeria','Angola','Benin','Cameroon','Cape Verde',
'Democratic Republic of the Congo','Egypt','Equatorial Guinea','Gabon''Gambia','Ghana','Guinea','Guinea-Bissau','Ivory Coast''Liberia','Libya''Mauritania',
'Morocco','Namibia','Nigeria','Republic of the Congo','São Tomé and Príncipe','Senegal','Sierra Leone','South Africa','Togo','Tunisia','Bouvet Island',
'Canary Islands','Madeira','Saint Helena, Ascension and Tristan da Cunha','Western Sahara','Cyprus','Egypt','Georgia','Israel','Lebanon','Russia','Syria','Turkey',
'Abkhazia','Gaza Strip','Northern Cyprus' ,'Argentina','Brazil','Chile','Colombia','Guyana','Suriname','Uruguay','Venezuela','Falkland Islands' ,'French Guiana' ,'South Georgia' 
'Belize','Canada','Costa Rica','Guatemala','Honduras','Mexico','Nicaragua','Panama','USA','Bermuda' ,'Greenland' ,'Saint Pierre and Miquelon',
'Antigua and Barbuda','Bahamas','Barbados','Cuba','Dominica','Dominican Republic','Grenada','Haiti','Jamaica','Saint Kitts and Nevis','Saint Lucia','Saint Vincent and the Grenadines',
'Trinidad and Tobago','Aruba','Bonaire','British Virgin Islands ','Cayman Islands','Curaçao','Guadeloupe','Martinique','Montserrat','Saba','Puerto Rico','Saint Barthélemy',
'Sint Maarten','Saint Martin','int Eustatius','Turks and Caicos Islands','United States Virgin Islands' ,'Anguilla' ]
Atlantic_ocean = [element.lower()  for element in Atlantic_ocean] 
Indian_ocean = ['South Africa','Mozambique','Madagascar','French Southern and Antarctic Lands','France','Mauritius','Comoros','Tanzania','Seychelles','Kenya','Somalia','Somaliland',
'Djibouti','Eritrea','Sudan','Egypt','Israel','Jordan','Palestine' ,'Saudi Arabia','Yemen','Oman','United Arab Emirates','Qatar','Bahrain','Kuwait','Iraq',
'Iran','Pakistan','India','Maldives','British Indian Ocean Territory','Sri Lanka','Bangladesh','Burma','Thailand','Malaysia','Singapore','Indonesia','Cocos Islands','Timor-Leste', 'Mayotte', 'Reunion', 'andian ocean']
Indian_ocean = [element.lower()  for element in Indian_ocean] 

table_shark.country = [element.lower()  for element in table_shark.country] 
liste = []
for t in table_shark.country:
    if t in Pacific_ocean:
        liste.append('pacific ocean')
    elif t in Atlantic_ocean:
        liste.append('atlantic ocean')
    elif t in Indian_ocean:
        liste.append('indian ocean')
    else:
        liste.append(t)   

In [23]:
#19 Update the column ( drop the old one and create the new)
n = table_shark.columns[4]
table_shark.drop(n, axis = 1, inplace = True)

In [24]:
table_shark[n] = liste #(create the new column)

In [25]:
#20 Drop the unique values in the "country" column
table_shark = table_shark[table_shark.country.duplicated(keep=False)]

In [26]:
table_shark.country.value_counts()

atlantic ocean    988
pacific ocean     376
indian ocean       29
Name: country, dtype: int64

# Attribute "type"

In [27]:
#21 Drop all the line that have an invalid 'type'

table_shark.type= table_shark.type.replace('Invalid',np.nan) 
table_shark = table_shark.dropna()
table_shark= table_shark.reset_index(drop=True)        


In [28]:
table_shark

,id,date,year,type,area,location,activity,fatalyn,species,source,time,injury,country
0,2016.09.11,11-Sep-16,2016,Unprovoked,Florida,"Ponte Vedra, St. Johns County",Wading,N,3' to 4' shark,"News4Jax, 9/11/2016",earlyafternoon,injury,atlantic ocean
1,2016.09.07,07-Sep-16,2016,Unprovoked,Hawaii,"Makaha, Oahu",Swimming,N,"Tiger shark, 10?","Hawaii News Now, 9/7/2016",earlyafternoon,injury,atlantic ocean
2,2016.08.29.b,29-Aug-16,2016,Unprovoked,Florida,"New Smyrna Beach, Volusia County",Surfing,N,"Bull shark, 6'","News Channel 8, 8/30/16",earlyafternoon,injury,atlantic ocean
3,2016.07.29,29-Jul-16,2016,Unprovoked,Alicante Province,Arenales del Sol,Swimming,N,Blue shark,"Informacion.es, 7/29/2016",morning,injury,atlantic ocean
4,2016.07.07.b,07-Jul-16,2016,Provoked,Massachusetts,"Off Gloucester, Essec County",Fishing,N,dogfish shark,Salem News 7/8/2016,morning,death,atlantic ocean
5,2016.07.04,04-Jul-16,2016,Provoked,Queensland,Palm Cove,Fishing,N,Tawny nurse shark,"Cairns Post, 7/9/2016",night,death,pacific ocean
6,2016.06.21.b,21-Jun-16,2016,Unprovoked,South Carolina,"North Myrtle Beach, Horry County",Floating,N,3' to 5' shark,"C. Creswell, GSAF",earlyafternoon,injury,atlantic ocean
7,2016.06.15.b,15-Jun-16,2016,Unprovoked,Hawaii,"Kalapaki Beach, Kauai",Surfing,N,3' to 4' shark,"West Hawaii Today, 6/16/2016",earlymorning,injury,atlantic ocean
8,2016.06.14,14-Jun-16,2016,Unprovoked,Texas,"Pirates Beach, Galveston",Floating in tube,N,3' to 4' shark,"Click2Houston, 6/14/2016",lateafternoon,injury,atlantic ocean
9,2016.06.11,11-Jun-16,2016,Unprovoked,North Carolina,"Atlantic Beach, Emerald Isle, Carteret County",Standing,N,3' shark,"C. Creswell, GSAF",earlyafternoon,injury,atlantic ocean


In [29]:
#22 The types: Boat, Sea disaster, Boating, can be considered as Unprovoked to simplify the column
for t in table_shark.type:
    if t in ['Boat', 'Sea Disaster','Boating']:
        table_shark = table_shark.replace(t,'Unprovoked') 

        

In [30]:
table_shark

,id,date,year,type,area,location,activity,fatalyn,species,source,time,injury,country
0,2016.09.11,11-Sep-16,2016,Unprovoked,Florida,"Ponte Vedra, St. Johns County",Wading,N,3' to 4' shark,"News4Jax, 9/11/2016",earlyafternoon,injury,atlantic ocean
1,2016.09.07,07-Sep-16,2016,Unprovoked,Hawaii,"Makaha, Oahu",Swimming,N,"Tiger shark, 10?","Hawaii News Now, 9/7/2016",earlyafternoon,injury,atlantic ocean
2,2016.08.29.b,29-Aug-16,2016,Unprovoked,Florida,"New Smyrna Beach, Volusia County",Surfing,N,"Bull shark, 6'","News Channel 8, 8/30/16",earlyafternoon,injury,atlantic ocean
3,2016.07.29,29-Jul-16,2016,Unprovoked,Alicante Province,Arenales del Sol,Swimming,N,Blue shark,"Informacion.es, 7/29/2016",morning,injury,atlantic ocean
4,2016.07.07.b,07-Jul-16,2016,Provoked,Massachusetts,"Off Gloucester, Essec County",Fishing,N,dogfish shark,Salem News 7/8/2016,morning,death,atlantic ocean
5,2016.07.04,04-Jul-16,2016,Provoked,Queensland,Palm Cove,Fishing,N,Tawny nurse shark,"Cairns Post, 7/9/2016",night,death,pacific ocean
6,2016.06.21.b,21-Jun-16,2016,Unprovoked,South Carolina,"North Myrtle Beach, Horry County",Floating,N,3' to 5' shark,"C. Creswell, GSAF",earlyafternoon,injury,atlantic ocean
7,2016.06.15.b,15-Jun-16,2016,Unprovoked,Hawaii,"Kalapaki Beach, Kauai",Surfing,N,3' to 4' shark,"West Hawaii Today, 6/16/2016",earlymorning,injury,atlantic ocean
8,2016.06.14,14-Jun-16,2016,Unprovoked,Texas,"Pirates Beach, Galveston",Floating in tube,N,3' to 4' shark,"Click2Houston, 6/14/2016",lateafternoon,injury,atlantic ocean
9,2016.06.11,11-Jun-16,2016,Unprovoked,North Carolina,"Atlantic Beach, Emerald Isle, Carteret County",Standing,N,3' shark,"C. Creswell, GSAF",earlyafternoon,injury,atlantic ocean


In [31]:
#23 Export the dataframe to csv file

table_shark.to_csv('data_shark.csv', sep='\t')